# Test read speeds for H5coro and xarray for IS2 and GEDI
Author: Tasha Snow

In [1]:
%pip install --quiet "h5coro>=0.0.7"

Note: you may need to restart the kernel to use updated packages.


In [1]:
%matplotlib widget

import numpy as np
import os
import matplotlib.pyplot as plt
import earthaccess
import xarray as xr
import h5py
from xarray.backends.api import open_datatree
from h5coro import h5coro, s3driver
import geopandas as gpd
import fsspec
import s3fs

In [2]:
# Authenticate for accessing NASA data (MODIS)
auth = earthaccess.login(strategy="netrc")

# If we are not authenticated
if not auth.authenticated:
    # Ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

In [3]:
bbox = (-122, 39.3, -120, 40) # west, south, east, north
start_dt = '2020-08-01'
end_dt = '2020-10-31'

In [4]:
# Gather all files from search location and time
results = earthaccess.search_data(
    concept_id='C2142776747-LPCLOUD',
    bounding_box=bbox,
    temporal=(start_dt, end_dt),
    cloud_hosted=True
)
print (f'{len(results)} TOTAL granules')

30 TOTAL granules


In [5]:
paths = earthaccess.open(results)

QUEUEING TASKS | :   0%|          | 0/30 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/30 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/30 [00:00<?, ?it/s]

In [13]:
# Get S3 credentials for accessing the LPDAAC data
s3_creds = auth.get_s3_credentials(daac="LPDAAC")

# Extract the credentials
aws_access_key_id = s3_creds['accessKeyId']
aws_secret_access_key = s3_creds['secretAccessKey']
aws_session_token = s3_creds['sessionToken']

# Initialize S3FileSystem with the obtained credentials
fs = s3fs.S3FileSystem(key=aws_access_key_id, 
                       secret=aws_secret_access_key, 
                       token=aws_session_token)

In [29]:
# # Explore the structure of the file
# def explore_h5py_group(group, indent=0):
#     for key in group.keys():
#         item = group[key]
#         if isinstance(item, h5py.Group):
#             print("  " * indent + f"Group: {key}")
#             explore_h5py_group(item, indent + 1)
#         elif isinstance(item, h5py.Dataset):
#             print("  " * indent + f"Dataset: {key} - Shape: {item.shape} - Type: {item.dtype}")
#             print("  " * indent + f"  Attributes:")
#             for attr_name, attr_value in item.attrs.items():
#                 print("  " * (indent + 1) + f"{attr_name}: {attr_value}")
#         else:
#             print("  " * indent + f"Unknown item: {key}")

In [ ]:
# # ds = xr.open_dataset(paths[0], engine='h5netcdf')  # You might need to specify the engine depending on the file type

# # Open the file
# with h5py.File(paths[0], 'r') as f:
#     # Start exploring from the root of the file
#     explore_h5py_group(f)

In [39]:
# results[0].data_links(in_region=True)

['s3://lp-prod-protected/GEDI02_B.002/GEDI02_B_2020216204134_O09309_02_T02435_02_003_01_V002/GEDI02_B_2020216204134_O09309_02_T02435_02_003_01_V002.h5']

## Test read speeds GEDI

In [68]:
%%timeit

# Read using h5py + xarray using an optimization technique (doesn't improve speed)
with h5py.File(paths[0],rdcc_nbytes=4*1024*1024) as f:
    lat = f['BEAM0001/geolocation/lat_lowestmode'][...]
    lon = f['BEAM0001/geolocation/lon_lowestmode'][...]
    cover = f["BEAM0001/cover"][...]
    f. close()

xr_cover = xr.DataArray(data=cover,
                        coords={'lat':(['x'],lat),
                                'lon':(['x'],lon)},
                        dims = ['x'])

1.7 s ± 160 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit

# Use s3fs to open the HDF5 file
with fs.open(paths[0], 'rb') as f:
    # Open the HDF5 file using h5py
    with h5py.File(f, 'r') as hdf:
        # Example: List all groups
        # print("Keys: %s" % hdf.keys())
        # Example: Access a specific dataset
        lat = hdf['BEAM0001/geolocation/lat_lowestmode'][...]
        lon = hdf['BEAM0001/geolocation/lon_lowestmode'][...]
        cover = hdf["BEAM0001/cover"][...]

        # Create a Pandas DataFrame
        df = pd.DataFrame({
            'latitude': lat,
            'longitude': lon,
            'cover': cover
        })

        # Convert to a GeoPandas DataFrame
        geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry)

        # Set the coordinate reference system (CRS) - adjust as needed
        gdf.set_crs(epsg=4326, inplace=True)

2.37 s ± 83.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
# Initialize the H5Coro object with the S3 driver and credentials
h5obj = h5coro.H5Coro(paths[0].details['name'], h5coro.s3driver.S3Driver, 
                      errorChecking=True, verbose=False, 
                      credentials=s3_creds, multiProcess=False)

# Define the variables you want to read
variables = ['BEAM0001/geolocation/lat_lowestmode',
             'BEAM0001/geolocation/lon_lowestmode',
             'BEAM0001/cover']

# Read the data using h5coro
data = h5obj.readDatasets(variables, block=True, enableAttributes=False)

# Extract the lat, lon, and cover data
lat = data['BEAM0001/geolocation/lat_lowestmode']
lon = data['BEAM0001/geolocation/lon_lowestmode']
cover = data['BEAM0001/cover']

# Create a Pandas DataFrame
df = pd.DataFrame({
    'latitude': lat,
    'longitude': lon,
    'cover': cover
})

# Convert to a GeoPandas DataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set the coordinate reference system (CRS) - adjust as needed
gdf.set_crs(epsg=4326, inplace=True)

1.58 s ± 28.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
%%timeit

# Read GEDI using h5coro + xarray
h5obj = h5coro.H5Coro(paths[0].details['name'], h5coro.s3driver.S3Driver, 
                      errorChecking=True, verbose=False, 
                      credentials=s3_creds, multiProcess=False)
variables = ['BEAM0001/geolocation/lat_lowestmode',
             'BEAM0001/geolocation/lon_lowestmode',
             'BEAM0001/cover']
data = h5obj.readDatasets(variables, block=True, enableAttributes=False)
# for variable in data:
#     print(f'{variable}: {data[variable][0:10]}')

# Convert to dataArray
xr_cover_coro = xr.DataArray(data=data['BEAM0001/cover'],
                        coords={'lat':(['x'],data['BEAM0001/geolocation/lat_lowestmode']),
                                'lon':(['x'],data['BEAM0001/geolocation/lon_lowestmode'])},
                        dims = ['x'])

824 ms ± 55.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# # Convert the data to an xarray.Dataset
# ds = xr.Dataset(
#     {
#         "shot_number": (["shot"], data['BEAM0001/geolocation/shot_number']),
#         "lat_lowestmode": (["shot"], data['BEAM0001/geolocation/lat_lowestmode']),
#         "lon_lowestmode": (["shot"], data['BEAM0001/geolocation/lon_lowestmode']),
#         "cover": (["shot", "z"], data['BEAM0001/cover'])  # Adjust dimensions as needed
#     }
# )

# # Optionally, add any coordinates or attributes if needed
# ds = ds.assign_coords(
#     shot=("shot", range(len(data['BEAM0101/geolocation/shot_number'])))
# )

# # Add global attributes or dataset-specific metadata if required
# ds.attrs["title"] = "GEDI Data Converted from HDF5 to xarray"

In [73]:
xr_cover

<xarray.DataArray (x: 108506)> Size: 434kB
array([-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        8.2312271e-02,  3.1103685e-01,  7.6448150e-02], dtype=float32)
Coordinates:
    lat      (x) float64 868kB 24.49 24.49 24.49 24.49 ... 51.82 51.82 51.82
    lon      (x) float64 868kB -137.8 -137.8 -137.8 ... -72.53 -72.53 -72.53
Dimensions without coordinates: x

In [71]:
xr_cover_coro

<xarray.DataArray (x: 108506)> Size: 434kB
array([-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        8.2312271e-02,  3.1103685e-01,  7.6448150e-02], dtype=float32)
Coordinates:
    lat      (x) float64 868kB 24.49 24.49 24.49 24.49 ... 51.82 51.82 51.82
    lon      (x) float64 868kB -137.8 -137.8 -137.8 ... -72.53 -72.53 -72.53
Dimensions without coordinates: x

## Time opening of IS2 data

In [10]:
# Gather all files from search location and time
results = earthaccess.search_data(
    concept_id='C2613553260-NSIDC_CPRD',
    bounding_box=bbox,
    temporal=(start_dt, end_dt),
    cloud_hosted=True
)
print (f'{len(results)} TOTAL granules')

18 TOTAL granules


In [11]:
paths = earthaccess.open(results)

QUEUEING TASKS | :   0%|          | 0/18 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/18 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/18 [00:00<?, ?it/s]

In [12]:
variables = ['/gt1r/land_segments', '/gt1l/land_segments', '/gt2r/land_segments']

In [100]:
%%timeit

# Open IS2 data in xarray
xr.open_dataset(paths[0],
                group=variables[0],
                engine='h5netcdf',
                backend_kwargs={'phony_dims':'access'},
)

36.3 s ± 1.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
***Not working yet
# Open IS2 data using h5coro + xarray
h5obj = h5coro.H5Coro(paths[0].details['name'], s3driver.S3Driver, errorChecking=True, verbose=False, credentials=s3_creds, multiProcess=False)
data = h5obj.readDatasets(variables[0], block=True, enableAttributes=False)# Read the datasets from the group using h5coro

# Convert the promise object to a dictionary (assuming promise is a dictionary-like object)
data_dict = {var.split('/')[-1]: data[var] for var in variables}

# Create an xarray.Dataset from the data
ds = xr.Dataset(data_dict)